## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia


In this lab, you will be performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches


In [1]:
import sys
import requests
import unicodedata
import pandas as pd

from bs4 import BeautifulSoup

In [39]:
# define helper functions for scraping

def date_time(table_cells):
    return [date_time.strip() for date_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    return " ".join([booster_version for i, booster_version in enumerate(table_cells.strings) if i%2==0][0:-1])

def landing_status(table_cells):
    return [i for i in table_cells.strings][0]

def get_mass(table_cells):
    mass=unicodedata.normalize('NFKD',table_cells.text).strip()
    if mass:
        # mass.find('kg')
        new_mass=mass[0:mass.find('kg')+2]

    else:
        new_mass=0
    return new_mass

def extract_column_from_header(row):
    if(row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()

    column_name=' '.join(row.contents)

    if not(column_name.strip().isdigit()):
        column_name=column_name.strip()
        return column_name

In [3]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

Task1


In [4]:
#HTTP GET method to request to page as an HTTP response

response=requests.get(static_url)

In [6]:
# create a beautiful soup object to parse
soup = BeautifulSoup(response.text, 'html.parser')

In [ ]:
soup.title  #confirm if object was created properly

<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>

Task2

In [9]:
html_tables=soup.find_all('table')
print(len(html_tables))


26


In [11]:
first_launch_table=html_tables[2]

In [13]:
flt_th=first_launch_table.find_all('th')
print(flt_th)

[<th scope="col">Flight No.
</th>, <th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>, <th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11"><span class="cite-bracket">[</span>b<span class="cite-bracket">]</span></a></sup>
</th>, <th scope="col">Launch site
</th>, <th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12"><span class="cite-bracket">[</span>c<span class="cite-bracket">]</span></a></sup>
</th>, <th scope="col">Payload mass
</th>, <th scope="col">Orbit
</th>, <th scope="col">Customer
</th>, <th scope="col">Launch<br/>outcome
</th>, <th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th>, <th rowspan="2" s

In [24]:
column_names=[]

# find_all 'th' in first_launch_table
flt_th=first_launch_table.find_all('th')

for i in flt_th:
    column_name= extract_column_from_header(i)
    if column_name !=None and len(column_name)>0:
        column_names.append(column_name)

        

In [25]:
column_names

['Flight No.',
 'Date and time ( )',
 'Launch site',
 'Payload',
 'Payload mass',
 'Orbit',
 'Customer',
 'Launch outcome']

Task3

In [27]:
#create a df by parsing the html tables

launch_dict=dict.fromkeys(column_names)

del launch_dict['Date and time ( )']

In [41]:
# initialize launch_dict with each value to be an empty list
launch_dict = {key: [] for key in column_names}

# new columns
launch_dict.update({'Version Booster':[],'Booster landing':[],'Date':[],'Time':[]})

In [34]:
print(launch_dict['Flight No.'])

[]


In [44]:
# Fill up launch_dict with launch records extracted from table rows
extracted_row = 0

# Extract each table 
for table_number, table in enumerate(soup.find_all('table', "wikitable plainrowheaders collapsible")):
    # Get table rows 
    for rows in table.find_all("tr"):
        # Check if first table heading is a number corresponding to a launch number
        if rows.th:
            if rows.th.string:
                flight_number = rows.th.string.strip()
                flag = flight_number.isdigit()
        else:
            flag = False

        # Get table elements 
        row = rows.find_all('td')

        # If it is a valid number, save cells in a dictionary
        if flag:
            extracted_row += 1

            # Flight Number value
            launch_dict['Flight No.'].append(flight_number)

            # Extract Date and Time
            datatimelist = date_time(row[0])
            launch_dict['Date'].append(datatimelist[0].strip(','))  # Date
            launch_dict['Time'].append(datatimelist[1])  # Time

            # Booster version
            bv = booster_version(row[1])
            if not bv:
                bv = row[1].a.string if row[1].a else row[1].text.strip()
            launch_dict['Version Booster'].append(bv)

            # Launch Site
            launch_site = row[2].a.string if row[2].a else row[2].text.strip()
            launch_dict['Launch site'].append(launch_site)

            # Payload
            payload = row[3].a.string if row[3].a else row[3].text.strip()
            launch_dict['Payload'].append(payload)

            # Payload Mass
            payload_mass = get_mass(row[4])
            launch_dict['Payload mass'].append(payload_mass)

            # Orbit
            orbit = row[5].a.string if row[5].a else row[5].text.strip()
            launch_dict['Orbit'].append(orbit)

            # Customer
            customer = row[6].a.string if row[6].a else row[6].text.strip()
            launch_dict['Customer'].append(customer)

            # Launch outcome
            launch_outcome = list(row[7].strings)[0] if row[7].strings else row[7].text.strip()
            launch_dict['Launch outcome'].append(launch_outcome)

            # Booster landing
            booster_landing = landing_status(row[8])
            launch_dict['Booster landing'].append(booster_landing)


In [45]:
df=pd.DataFrame({key:pd.Series(value) for key , value in launch_dict.items()})

In [46]:
df.to_csv('spacex_web_scraped.csv',index=False)